<a href="https://colab.research.google.com/github/Addi-11/Depression-chatbot/blob/temporary/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
import collections

import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding, CuDNNLSTM, GlobalMaxPooling1D, GlobalAveragePooling1D, CuDNNGRU
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import helper

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import re

In [ ]:
import pandas
df=pandas.read_csv('data.csv',sep=None, error_bad_lines=False)
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  
Skipping line 3800: unexpected end of data


,title,response
0,It kills me to see others in worse positions t...,"I don’t know who you are, but I needed this an..."
1,"I don’t know who you are, but I needed this an...",Its no problem. I just know what its like. I'v...
2,Summer depression,[deleted]
3,[deleted],I think it might be the hot weather for me... ...
4,[Chronically depressed 3+ years] I feel pathet...,It’s nice when people give us compliments. We ...


In [ ]:
df['title'][0]

"It kills me to see others in worse positions than I am. You matter. You all matter and it doesn't matter what your depression or suicidal thoughts tell you."

In [ ]:
df['response'][0]

'I don’t know who you are, but I needed this and I thank you. I’ve been stuck ina runt for a couple months now. Wanting to just die and let it all be over with. But after reading it, I have a different mindset. Once again thank you kind stranger'

In [ ]:
df=df[(df.title!='[deleted]') & (df.response!='[deleted]')]

In [ ]:
df.shape

(3750, 2)

In [ ]:
df.head()

,title,response
0,It kills me to see others in worse positions t...,"I don’t know who you are, but I needed this an..."
1,"I don’t know who you are, but I needed this an...",Its no problem. I just know what its like. I'v...
4,[Chronically depressed 3+ years] I feel pathet...,It’s nice when people give us compliments. We ...
5,Sick of being sad and lonely,"Dude, I'm terribly sorry for how things have h..."
6,"Dude, I'm terribly sorry for how things have h...",This is probably some of the best advice I hav...


In [ ]:
!pip install contractions

In [ ]:
import contractions

In [ ]:
df['title']=df['title'].apply(lambda x:x.replace(x,contractions.fix(x)))

In [ ]:
df['response']=df['response'].apply(lambda x:x.replace(x,contractions.fix(x)))

In [ ]:
df['title']=df['title'].str.replace("[^a-zA-Z#.,]"," ")

In [ ]:
df['response']=df['response'].str.replace("[^a-zA-Z#.,]"," ")

In [ ]:
df.head()

,title,response
0,It kills me to see others in worse positions t...,"I do not know who you are, but I needed this a..."
1,"I do not know who you are, but I needed this a...",Its no problem. I just know what its like. I h...
4,Chronically depressed years I feel pathet...,it is nice when people give us compliments. We...
5,Sick of being sad and lonely,"Dude, I am terribly sorry for how things have ..."
6,"Dude, I am terribly sorry for how things have ...",This is probably some of the best advice I hav...


In [ ]:
df['title']=df['title'].apply(lambda x : x.lower().strip())
df['response']=df['response'].apply(lambda x : x.lower().strip())

In [ ]:
df.head()

,title,response
0,it kills me to see others in worse positions t...,"i do not know who you are, but i needed this a..."
1,"i do not know who you are, but i needed this a...",its no problem. i just know what its like. i h...
4,chronically depressed years i feel patheti...,it is nice when people give us compliments. we...
5,sick of being sad and lonely,"dude, i am terribly sorry for how things have ..."
6,"dude, i am terribly sorry for how things have ...",this is probably some of the best advice i hav...


In [ ]:
!pip install autocorrect
from autocorrect import Speller

In [ ]:
spell=Speller(lang='en')
df['title']=df['title'].apply(lambda x: x.replace(x,spell(x)))

In [ ]:
spell=Speller(lang='en')
df['response']=df['response'].apply(lambda x: x.replace(x,spell(x)))

In [ ]:
def preprocess_sentence(w):

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    
    return w

In [ ]:
df['title'] = df['title'].progress_apply(lambda x: preprocess_sentence(x))
df['response'] = df['response'].progress_apply(lambda x: preprocess_sentence(x))
df.head()


100%|██████████| 3750/3750 [00:00<00:00, 23034.59it/s]

100%|██████████| 3750/3750 [00:00<00:00, 15329.88it/s]


,title,response
0,<start> it kills me to see others in worse pos...,"<start> i do not know who you are , but i need..."
1,"<start> i do not know who you are , but i need...",<start> its no problem . i just know what its ...
4,<start> chronically depressed years i feel pat...,<start> it is nice when people give us complim...
5,<start> sick of being sad and lonely <end>,"<start> dude , i am terribly sorry for how thi..."
6,"<start> dude , i am terribly sorry for how thi...",<start> this is probably some of the best advi...


In [ ]:
titles=df['title'].values
import string
title=[]
for i in range(len(titles)):
  title.append(titles[i].lower().strip())

In [ ]:
responses=df['response'].values
response=[]
for i in range(len(responses)):
    response.append(responses[i].lower().strip())

In [ ]:
for i in range(2):
  print('Person 1 : ', title[i])
  print('Person 2 : ', response[i])

Person 1 :  <start> it kills me to see others in worse positions than i am . you matter . you all matter and it does not matter what your depression or suicidal thoughts tell you . <end>
Person 2 :  <start> i do not know who you are , but i needed this and i thank you . i have been stuck in runt for a couple months now . wanting to just die and let it all be over with . but after reading it , i have a different mindset . once again thank you kind stranger <end>
Person 1 :  <start> i do not know who you are , but i needed this and i thank you . i have been stuck in runt for a couple months now . wanting to just die and let it all be over with . but after reading it , i have a different mindset . once again thank you kind stranger <end>
Person 2 :  <start> its no problem . i just know what its like . i have been in this rut for a year , and i hate how many people are in the same , if not a worse position . do not kill yourself . all the things that made you happy in the past , that can s

In [ ]:
title_counter = collections.Counter([word for sentence in title for word in sentence.split()])
response_counter = collections.Counter([word for sentence in response for word in sentence.split()])

print('{} Title sentences.'.format(len(title)))
print('{} Title words.'.format(len([word for sentence in title for word in sentence.split()])))
print('{} unique Title words.'.format(len(title_counter)))
print('10 Most common words in the Title:')
print('"' + '" "'.join(list(zip(*title_counter.most_common(10)))[0]) + '"')
print()
print('{} Response sentences.'.format(len(response)))
print('{} Response words.'.format(len([word for sentence in response for word in sentence.split()])))
print('{} unique Response words.'.format(len(response_counter)))
print('10 Most common words in the Response dataset:')
print('"' + '" "'.join(list(zip(*response_counter.most_common(10)))[0]) + '"')

3750 Title sentences.
180862 Title words.
8766 unique Title words.
10 Most common words in the Title:
"." "i" "to" "," "you" "and" "a" "<start>" "<end>" "the"

3750 Response sentences.
291977 Response words.
10643 unique Response words.
10 Most common words in the Response dataset:
"." "i" "," "to" "you" "and" "the" "a" "is" "it"


In [ ]:
def tokenize(x):
  t=Tokenizer()
  t.fit_on_texts(x)
  return t.texts_to_sequences(x),t

In [ ]:
# Test Case for tokenize function
text_sentences=['My name is Sahil.','I am pursuing btech from nith.']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(temp, temp1)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'my': 1, 'name': 2, 'is': 3, 'sahil': 4, 'i': 5, 'am': 6, 'pursuing': 7, 'btech': 8, 'from': 9, 'nith': 10}

Sequence 1 in x
  Input:  My name is Sahil.
  Output: [[1, 2, 3, 4], [5, 6, 7, 8, 9, 10]]
Sequence 2 in x
  Input:  I am pursuing btech from nith.
  Output: <keras_preprocessing.text.Tokenizer object at 0x7f9624f05860>


In [ ]:
def pad(x,length=None):
  if length is None:
    length=max([len(sentence) for sentence in x])
  return pad_sequences(x,maxlen=length,padding="post")
  

In [ ]:
#test for pad
test_pad=pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 3 4]
  Output: [1 2 3 4 0 0]
Sequence 2 in x
  Input:  [ 5  6  7  8  9 10]
  Output: [ 5  6  7  8  9 10]


In [ ]:
#function to use tokenize and pad function 
def preprocess(x, y):

    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [ ]:
preproc_title, preproc_response, title_tokenizer, response_tokenizer =\
    preprocess(title, response)

max_title_sequence_length = preproc_title.shape[1]
max_response_sequence_length = preproc_response.shape[1]
title_vocab_size = len(title_tokenizer.word_index)
response_vocab_size = len(response_tokenizer.word_index)

print('Data Preprocessed')
print("Max Title sentence length:", max_title_sequence_length)
print("Max Response sentence length:", max_response_sequence_length)
print("Title vocabulary size:", title_vocab_size)
print("Response vocabulary size:", response_vocab_size)

Data Preprocessed
Max Title sentence length: 1619
Max Response sentence length: 1619
Title vocabulary size: 8762
Response vocabulary size: 10639
